## Imports

In [1]:
from pulp import *
import googlemaps
import pandas as pd

In [2]:
df_unidades = pd.read_csv(r'..\2. Obtenção da matriz de distâncias\Matriz de distâncias final.csv').drop(columns=['Unnamed: 0'])
df_centros = pd.read_csv(r'..\2. Obtenção da matriz de distâncias\Centros.csv')

In [3]:
unidades = df_unidades['Coordenadas'].to_list()
CDs = df_centros['centros'].to_list()

oferta = df_centros[['centros', 'Oferta']].set_index('centros').to_dict()['Oferta']
demanda = df_unidades[['Coordenadas', 'Demanda']].set_index('Coordenadas').to_dict()['Demanda']

df = df_unidades.drop(columns = ['UBS ou ponto de vacinação', 'Nome Google Maps', 'Demanda']).set_index('Coordenadas')
df = df.to_dict()

## Modeling

In [4]:
prob = LpProblem("THE_P_MEDIAN_PROBLEM", LpMinimize)

In [5]:
p = 5

In [6]:
tarifa = 0.01 # R$/KM*1u

In [7]:
x = LpVariable.dicts('X_%s_to_%s', (CDs,unidades), 
    cat = 'Binary', 
    lowBound = 0, 
    upBound = 1)

In [8]:
y = LpVariable.dicts('Y_%s', (CDs),
    cat = 'Binary', 
    lowBound = 0, 
    upBound = 1)

#### Objective function

In [9]:
prob += sum(x[i][j]*df[i][j]*demanda[j]*tarifa for i in CDs for j in unidades), "Custo total estimado"

#### Constraints

In [10]:
# cada unidade deve receber vacinas de um, e apenas um, CD
for j in unidades:
        prob += sum(x[i][j] for i in CDs) == 1

In [11]:
# a quantodade de CDs abertos é igual a p
prob += lpSum(y[j] for j in CDs) == p

In [12]:
# apenas CDs abertos podem entregar vacinas
for i in CDs:
    for  j in unidades:
        prob += x[i][j]<= y[i]

In [13]:
# cada CD só pode atender no máximo x unidades
for i in CDs:
    prob += lpSum(x[i][j] for j in unidades) <= 40

In [14]:
# a soma das demandas das unidades para as quais determinado CD irá fazer entregas é menor que (ou igual) à oferta do CD
for i in CDs:
        prob += lpSum(demanda[j]*x[i][j] for j in unidades) <= oferta[i]*y[i]

In [15]:
prob

THE_P_MEDIAN_PROBLEM:
MINIMIZE
1152560.0*X_Hospital_Regional_da_Asa_Norte_to_(_15.4498015,__47.60926689999999) + 960960.0*X_Hospital_Regional_da_Asa_Norte_to_(_15.5236592,__47.6187935) + 1383600.0*X_Hospital_Regional_da_Asa_Norte_to_(_15.5389184,__48.1685104) + 773820.0*X_Hospital_Regional_da_Asa_Norte_to_(_15.5635984,__47.9353885) + 805760.0*X_Hospital_Regional_da_Asa_Norte_to_(_15.6027694,__47.6482851) + 809700.0*X_Hospital_Regional_da_Asa_Norte_to_(_15.6044882,__47.6602599) + 835940.0*X_Hospital_Regional_da_Asa_Norte_to_(_15.6123458,__47.643206) + 827880.0*X_Hospital_Regional_da_Asa_Norte_to_(_15.6153888,__47.6506249) + 719380.0*X_Hospital_Regional_da_Asa_Norte_to_(_15.6198199,__47.6803603) + 553240.0*X_Hospital_Regional_da_Asa_Norte_to_(_15.6208955,__47.82171779999999) + 772520.0*X_Hospital_Regional_da_Asa_Norte_to_(_15.6210428,__47.6580844) + 484740.0*X_Hospital_Regional_da_Asa_Norte_to_(_15.6227663,__47.8364748) + 917580.0*X_Hospital_Regional_da_Asa_Norte_to_(_15.6272893,__48.119

## Solving

In [16]:
prob.solve()

1

In [17]:
print(f'Total: R$ {prob.objective.value()/1000}')

Total: R$ 65965.06


In [18]:
for v in prob.variables():
    print(v.name , '=', v.varValue)
    #if v.varValue == 1:
        #print(v.name , '=', v.varValue)

X_Hospital_Regional_da_Asa_Norte_to_(_15.4498015,__47.60926689999999) = 0.0
X_Hospital_Regional_da_Asa_Norte_to_(_15.5236592,__47.6187935) = 0.0
X_Hospital_Regional_da_Asa_Norte_to_(_15.5389184,__48.1685104) = 0.0
X_Hospital_Regional_da_Asa_Norte_to_(_15.5635984,__47.9353885) = 0.0
X_Hospital_Regional_da_Asa_Norte_to_(_15.6027694,__47.6482851) = 0.0
X_Hospital_Regional_da_Asa_Norte_to_(_15.6044882,__47.6602599) = 0.0
X_Hospital_Regional_da_Asa_Norte_to_(_15.6123458,__47.643206) = 0.0
X_Hospital_Regional_da_Asa_Norte_to_(_15.6153888,__47.6506249) = 0.0
X_Hospital_Regional_da_Asa_Norte_to_(_15.6198199,__47.6803603) = 0.0
X_Hospital_Regional_da_Asa_Norte_to_(_15.6208955,__47.82171779999999) = 0.0
X_Hospital_Regional_da_Asa_Norte_to_(_15.6210428,__47.6580844) = 0.0
X_Hospital_Regional_da_Asa_Norte_to_(_15.6227663,__47.8364748) = 0.0
X_Hospital_Regional_da_Asa_Norte_to_(_15.6272893,__48.1195424) = 0.0
X_Hospital_Regional_da_Asa_Norte_to_(_15.6283193,__47.8376289) = 0.0
X_Hospital_Regional_d

X_Hospital_Regional_de_Taguatinga_to_(_15.6743594,__48.1515114) = 0.0
X_Hospital_Regional_de_Taguatinga_to_(_15.6772371,__48.1949629) = 0.0
X_Hospital_Regional_de_Taguatinga_to_(_15.6792755,__47.6483248) = 0.0
X_Hospital_Regional_de_Taguatinga_to_(_15.7026703,__47.75886060000001) = 0.0
X_Hospital_Regional_de_Taguatinga_to_(_15.7088082,__47.9134411) = 0.0
X_Hospital_Regional_de_Taguatinga_to_(_15.7109084,__47.876274) = 0.0
X_Hospital_Regional_de_Taguatinga_to_(_15.7209443,__47.88592209999999) = 0.0
X_Hospital_Regional_de_Taguatinga_to_(_15.7257082,__47.8739439) = 0.0
X_Hospital_Regional_de_Taguatinga_to_(_15.7386425,__47.7628634) = 0.0
X_Hospital_Regional_de_Taguatinga_to_(_15.7401874,__47.7633463) = 0.0
X_Hospital_Regional_de_Taguatinga_to_(_15.7403724,__47.7634136) = 0.0
X_Hospital_Regional_de_Taguatinga_to_(_15.7430148,__47.8916531) = 0.0
X_Hospital_Regional_de_Taguatinga_to_(_15.7431358,__47.89162049999999) = 0.0
X_Hospital_Regional_de_Taguatinga_to_(_15.7449298,__47.761381) = 0.0
X

In [19]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

df = pd.DataFrame(columns=['origem', 'destino'])
for v in prob.variables():
    #print(v.name , '=', v.varValue)
    if v.varValue == 1:
        #print(v.name , '=', v.varValue)
        split = v.name.split('_to_')
        split[0] = split[0].replace('X_', '')
        try:
            df = df.append({'origem': split[0], 'destino': split[1]}, ignore_index=True)
        except:
            pass

## Result

In [20]:
df['origem'] = df['origem'].replace('_', ' ', regex=True)
df['destino'] = df['destino'].replace('__', ' -', regex=True)
df['destino'] = df['destino'].replace('_', '-', regex=True)

In [25]:
df.style.set_sticky()

,origem,destino
0,Hospital Regional da Ceilândia,"(-15.5389184, -48.1685104)"
1,Hospital Regional da Ceilândia,"(-15.6272893, -48.1195424)"
2,Hospital Regional da Ceilândia,"(-15.6589014, -48.1930314)"
3,Hospital Regional da Ceilândia,"(-15.6698514, -48.2004701)"
4,Hospital Regional da Ceilândia,"(-15.6728185, -48.2003232)"
5,Hospital Regional da Ceilândia,"(-15.6743594, -48.1515114)"
6,Hospital Regional da Ceilândia,"(-15.6772371, -48.1949629)"
7,Hospital Regional da Ceilândia,"(-15.7851944, -48.1365487)"
8,Hospital Regional da Ceilândia,"(-15.7956911, -48.1309163)"
9,Hospital Regional da Ceilândia,"(-15.7983005, -48.1336503)"


In [22]:
df.to_csv('Optimized.csv')